# tsTools Online: MEaSUREs Training Data Collection

In [1]:
from ipywidgets import Button, HBox, VBox, Label, BoundedIntText, HTML, Dropdown, Layout
import bqplot
import datetime
import dateutil.parser
import ee
import ipywidgets
import ipywidgets as widgets
import IPython.display
import numpy as np
import pprint
import pandas as pd
import traitlets
import ipyleaflet
import geopandas
import bqplot.pyplot as plt
from dateutil import parser
import qgrid

from bqplot.interacts import BrushSelector, BrushIntervalSelector

import sqlite3
import os

from bqplot.interacts import BrushSelector, BrushIntervalSelector

# Google sheets API
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import ccd

# Turn off pandas warning
pd.options.mode.chained_assignment = None

# Configure the pretty printing output.
pp = pprint.PrettyPrinter(depth=4)



In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
# Authenticate to the Earth Engine servers, and initialize the ee module.
ee.Initialize()

In [4]:
# Functions to load collections, merge, filter and calculate statistics

# Filter collection by point and date
def collection_filtering(point, collection_name, year_range, doy_range):
    collection = ee.ImageCollection(collection_name)\
    .filterBounds(point)\
    .filter(ee.Filter.calendarRange(year_range[0], year_range[1], 'year'))\
    .filter(ee.Filter.dayOfYear(doy_range[0],doy_range[1]))
    return collection

# Landsat stack renamers, C1
def stack_renamer_l4_7_C1(img):
    band_list = ['B1', 'B2','B3','B4','B5','B7', 'B6','pixel_qa']
    name_list = ['BLUE','GREEN','RED','NIR','SWIR1','SWIR2','THERMAL','pixel_qa']
    return ee.Image(img).select(band_list).rename(name_list)

def stack_renamer_l8_C1(img):
    band_list = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10','pixel_qa'];
    name_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL','pixel_qa'];
    return ee.Image(img).select(band_list).rename(name_list);


# Cloud masking for C1, L4-L7. Operators capitalized to
# avoid confusing with internal Python operators
def cloud_mask_l4_7_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(66)).Or(pqa.eq(130))\
    .Or(pqa.eq(68)).Or(pqa.eq(132))
    return ee.Image(img).updateMask(mask)
  
# Cloud masking for C1, L8
def cloud_mask_l8_C1(img):
    pqa = ee.Image(img).select(['pixel_qa'])
    mask = (pqa.eq(322)).Or(pqa.eq(386)).Or(pqa.eq(324))\
    .Or(pqa.eq(388)).Or(pqa.eq(836)).Or(pqa.eq(900))
    return ee.Image(img).updateMask(mask)


# Calculate period statistics, seems to work but it's so ugly!
def calc_period_stats(yr_list, collection, reducer, stat_name, bands):
    def auxfnc(year):
        start = ee.Date(year)
        end = start.advance(step, 'year').advance(-1, 'day')

        return collection\
        .select(bands)\
        .filterDate(start, end)\
        .reduce(reducer)\
        .set({'system:time_start': start, 'system:time_end': end, 'statistic': stat_name})
    period_stats = yr_list.map(auxfnc)
    return period_stats

def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        dictarr.append(attr)

    return pd.DataFrame(dictarr)

def fc2dfgeo(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']
        dictarr.append(attr)
    df = geopandas.GeoDataFrame(dictarr)
    return df

def get_df(collection, coords, band):
    point = ee.Geometry.Point(coords)
    # Sample for a time series of values at the point.
    geom_values = collection.filterBounds(point).select(band).getRegion(geometry=point, scale=30)
    geom_values_list = ee.List(geom_values).getInfo()
    # Convert to a Pandas DataFrame.
    header = geom_values_list[0]
    data = pd.DataFrame(geom_values_list[1:], columns=header)
    data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
    data.set_index('time')
    data = data.sort_values('datetime')
    data = data[['id', 'datetime', band]]
    return data

def GetTileLayerUrl(ee_image_object):
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

# Load, filter and merge collections
def get_full_collection(coords, year_range, doy_range):
    point = ee.Geometry.Point(coords)
    l8_renamed = collection_filtering(point, 'LANDSAT/LC08/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l8_C1)
    l8_filtered1 = l8_renamed.map(cloud_mask_l8_C1)

    l7_renamed = collection_filtering(point, 'LANDSAT/LE07/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1);
    l7_filtered1 = l7_renamed.map(cloud_mask_l4_7_C1)

    l5_renamed = collection_filtering(point, 'LANDSAT/LT05/C01/T1_SR', year_range, doy_range)\
        .map(stack_renamer_l4_7_C1)
    l5_filtered1 = l5_renamed.map(cloud_mask_l4_7_C1)


    all_scenes = ee.ImageCollection((l8_filtered1.merge(l7_filtered1)).merge(l5_filtered1)).sort('system:time_start')
    return all_scenes

In [5]:
# PyCCD Functions
# Functions for running pyccd
def make_df_pyccd(collection, point):
    band_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    rename_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
    info = collection.getRegion(point, 30).getInfo()
    header = info[0]
    data = np.array(info[1:])
    iTime = header.index('time')
    time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]
    time_new = [t.toordinal() for t in (time)]
    iBands = [header.index(b) for b in band_list]
    yData = data[0:,iBands].astype(np.float)
    df = pd.DataFrame(data=yData, index=list(range(len(yData[:,0]))), columns=rename_list)
    df['time'] = time_new
    return df

def update_df(df, df2):
    df = df.append(df2)
    return df

def plot_pyccd(results, df, band, plotband, dates, yl, ylabel):
    mask = np.array(results['processing_mask']).astype(np.bool_)
    predicted_values = []
    prediction_dates = []
    break_dates = []
    start_dates = []

    for num, result in enumerate(results['change_models']):
        days = np.arange(result['start_day'], result['end_day'] + 1)
        prediction_dates.append(days)
        break_dates.append(result['break_day'])
        start_dates.append(result['start_day'])
        intercept = result[list(result.keys())[6+band]]['intercept']
        coef = result[list(result.keys())[6+band]]['coefficients']
    
        predicted_values.append(intercept + coef[0] * days +
                                coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
                                coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
                                coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))
    
    num_breaks = len(break_dates)

    # eric here
    break_y = [plotband[dates == i][0] for i in break_dates]

    #break_y = [0] * num_breaks
    break_dates_plot = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in break_dates]

    plot_dates = np.array([datetime.datetime.fromordinal(i) for i in (dates)])

    # Predicted curves
    all_dates = []
    all_preds = []
    for _preddate, _predvalue in zip(prediction_dates, predicted_values):
        all_dates.append(_preddate)
        all_preds.append(_predvalue)
        
    all_preds = [item for sublist in all_preds for item in sublist]
    all_dates = [item for sublist in all_dates for item in sublist]

    date_ord = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in all_dates]
    _x = np.array(date_ord, dtype='datetime64')
    _y = all_preds
    lc4.x = _x
    lc4.y = _y
    lc5.x = np.array(break_dates_plot, dtype='datetime64')
    lc5.y = break_y
    


In [6]:
class interface(object):
    """A class to hold interface for defining sample attributes"""
    
    # Set up database
    dbPath = os.getcwd() + '/sample_database'
    if not os.path.isfile(dbPath):
        conn = sqlite3.connect(dbPath)
        c = conn.cursor()
        c.execute('''CREATE TABLE sample
                 (id real, lat real, lon real, year1 real, year2 real, class text, class2 text, confidence real)''')
        print('Database created')
    
    # Create widgets
    # Years slider
    years = widgets.IntRangeSlider(value=[2012, 2015], min=1990, max=2018,
            step=1, description='Years:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    # Primary class selector
    lc = widgets.Dropdown(options=['Select Class','Vegetation', 'Barren', 'Water','Developed'],
                value='Select Class', description='Select Class', disabled=False)

    
    # Secondary class information
    second_class_drop = widgets.Dropdown(options=['Secondary Class Information'],
                value='Secondary Class Information', description='Secondary:',
                disabled=False)
    
    # Third tier class information
    third_class_drop = widgets.Dropdown(
                            options=['Additional Class Information'],
                            value='Additional Class Information',
                            #rows=10,
                            description='Third:',
                            disabled=False
                        )
    
    # Display current validity of sample
    valid = widgets.Valid(value=False, description='Not Saved', readout='')

    
    # Create button for checking validity
    validate = widgets.Button(value=False, description='Validate', disabled=False,
                button_style='', icon='check')
    
    # Save the current sample, only possible when validity has been approved
    save_button = widgets.Button(value=False, description='Save', disabled=False,
                button_style='')
    
    # Load everything
    load_button = widgets.Button(value=False, description='Load', disabled=False,
                button_style='')
    
    valid_load = widgets.Valid(value=False, description='Not Loaded', readout='')

    
    # Manually input class label
    #textClass = widgets.Text(value='', placeholder='', description='Enter Class',
    #                disabled=False)
    
    # Interpreters confidence
    confidence = widgets.IntSlider(value=0, min=0, max=3, step=1, description='Confidence:',
                    disabled=False, continuous_update=False, orientation='horizontal',
                    readout=True, readout_format='d')
    
    # Notes
    notes = widgets.Textarea(value='Enter any useful or interesting information about the sample.',
                    placeholder='Enter any useful or interesting information about the sample',
                    description='Notes:', layout=widgets.Layout(width='70%'), disabled=False)
    
    # Spreadsheet information
    #spreadsheet = widgets.Text(value='Google Spreadsheet Credential JSON',
    #                placeholder='Google Spreadsheet Credential JSON',
    spreadsheet = widgets.Text(value='MEaSUREs-9dc49ca010f9.json',
                    placeholder='MEaSUREs-9dc49ca010f9.json',
                    description='Credentials:',disabled=False,
                    continuous_update=True)
 
    #spreadName = widgets.Text(value='Google Spreadsheet Name',
    #                placeholder='Google Spreadsheet Name', description='Name:',
    spreadName = widgets.Text(value='MEaSUREs_ELB',
                    placeholder='MEaSUREs_ELB', description='Name:',
                    disabled=False, continuous_update=True)
    
    # Sample path users/parevalo_bu/sample_13_15_labels_pts
    #sampleWidget = widgets.Text(value='Path to sample feature collection',
    #            placeholder='Path to sample feature colleciton', description='Path:',
    sampleWidget = widgets.Text(value='users/parevalo_bu/sample_13_15_labels_pts',
                placeholder='users/parevalo_bu/sample_13_15_labels_pts', description='Sample:',
                                disabled=False, continuous_update=True)

    def __init__(self, path):
        #interface.pathDB
        interface.df = None
        interface.sheet = None
  
    
    def lc_clicked(selection):
        """ Generate secondary class selector after initial class is chosen """
        if selection.new == 'Vegetation':
            interface.second_class_drop.set_trait('options', ['Vegetation Growth Form','Cropland', 'Tree', 'Shrub','Herbaceous'])
        elif selection.new == 'Barren':
            interface.second_class_drop.set_trait('options', ['Barren Form','Sand/Beach', 'Rock', 'Ice/Snow'])
        elif selection.new == 'Developed':
            interface.second_class_drop.set_trait('options', ['Developed Form','Low Albedo', 'High Albedo'])
        elif selection.new == 'Water':
            interface.second_class_drop.set_trait('options', ['Water Form','Shore/Intertidal', 'Open Water'])
    
    def third_clicked(selection):
        """ Generate additional class selector after initial class is chosen """
        if selection.new == 'Herbaceous':
            interface.third_class_drop.set_trait('options', ['Additional Class Information','Moss/lichen', 'Natural grassland', 'Pastures','Grass/Lawn'])
        elif selection.new == 'Tree' or selection == 'Shrub':
            interface.third_class_drop.set_trait('options', ['Additional Class Information','Deciduous','Evergreen','Mixed'])
        elif selection.new == 'Cropland':
            interface.third_class_drop.set_trait('options', ['Only secondary information needed'])
        elif interface.lc.value == 'Barren' or interface.lc.value == 'Water' or interface.lc.value == 'Developed':
             interface.third_class_drop.set_trait('options', ['Only secondary information needed'])
    
    def check_val_status(selection):
        """ Check the validity of the current sample and change valid widget accordingly """
        selected_secondary_lc = False
        wrote_correct_lc = False
        if interface.second_class_drop.value != 'Secondary Class Information':
            selected_secondary_lc = True
        else:
            print("Must specify secondary class information!")
        if interface.lc.value.capitalize() == interface.textClass.value.capitalize():
            wrote_correct_lc = True
        if selected_secondary_lc and wrote_correct_lc:
            interface.valid.value = True
            interface.save_button.disabled = False
            #interface.valid.set_trait('value',True)
            #interface.save_button.set_trait('disabled',False)
  
    
    def save_sample(selection):
        """ Save the sample to the database """
        # TODO: This should be backed up somehow
        
        # Connect to the database
        conn = sqlite3.connect(interface.dbPath)
        c = conn.cursor()
        
        # Get everything in right format
        year1 = interface.years.value[0]
        year2 = interface.years.value[1]
        lcClass = interface.lc.value
        class2 = interface.second_class_drop.value
        class3 = interface.third_class_drop.value
        conf = interface.confidence.value

        # TODO: Read ID and geographic information from sample
        idSample = Plot_interface.current_id
        lat = Plot_interface.m.center[0]
        lon = Plot_interface.m.center[1]
        print(lat)
        print(lon)

        sampleInput = (idSample, lat, lon, year1, year2, lcClass, class2, conf)

        # Put sample information into database
        c.execute("""insert into sample
                  values {i}""".format(i=sampleInput))

        # Save (commit) the changes
        conn.commit()

        # Close the cursor
        c.close()
        
        # Change save validity state
        interface.valid.value = True
        interface.valid.description='Saved!'

        # Reset the buttons
        interface.years.set_trait('value',[2012, 2015])
        interface.lc.set_trait('value', 'Select Class')
        
        #interface.save_button.set_trait('disabled',True)
        
        # Save to drive
        sampleInputList = [idSample, lat, lon, year1, year2, lcClass, class2, class3, conf]
        interface.sheet.insert_row(sampleInputList, 2)

    def load_everything(sender):
        # Load Google Spreadsheet and GEE Feature Collection
        scope = ['https://spreadsheets.google.com/feeds' + ' ' +'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(interface.spreadsheet.value, scope)
        client = gspread.authorize(creds)
        interface.sheet = client.open(interface.spreadName.value).sheet1
        feat = ee.FeatureCollection(interface.sampleWidget.value)
        interface.df = fc2dfgeo(feat)
        interface.valid_load.value = True
        interface.valid_load.description='Loaded!'
        
    # Handle widget interactions
    # Call function check_val_status when button is clicked
    validate.on_click(check_val_status)    
    # Primary class selector action
    lc.observe(lc_clicked, 'value')
    # Secondary class selector action
    second_class_drop.observe(third_clicked, 'value')
    # Call function save_sample when clicking save button
    save_button.on_click(save_sample)
    
    # Load database and sample
    load_button.on_click(load_everything)


In [7]:
# Set global variables
year_range = [ 2000, 2018 ];
doy_range = [ 1, 365 ];
step = 1 #in years
dbPath = "/home/paulo/default_bu/notebooks/database/test.db"

In [8]:
# Load feature collection
testfeat = ee.FeatureCollection("users/parevalo_bu/sample_13_15_labels_pts")

# Get collection as geopandas df
df = fc2dfgeo(testfeat)

In [14]:
class Plot_interface(object):
    """Class to handle map and plot interaction"""

    # Declare class attributes
    current_id = 0
    pyccd_flag = False
    current_band = ''
    band_index = 4
    table = pd.DataFrame()
    band_list =['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2']
    year_range = [ 2000, 2018 ]
    doy_range = [ 1, 365 ]
    step = 1 #in years
    
    # Create widget controls
    next_pt = Button(value=False, description='Next point', disabled=False) 
    previous_pt = Button(value=False, description='Previous point', disabled=False)
    pyccd_button = Button(value=False, description='Run PyCCD', disabled=False)
    band_selector = Dropdown(options=['BLUE', 'GREEN', 'RED','NIR','SWIR1','SWIR2'], 
                             description='Select band', value=None)
    
    ylim = widgets.IntRangeSlider(value=[0, 4000], min=0, max=10000,
            step=500, description='YLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    xlim = widgets.IntRangeSlider(value=[1996, 2018], min=1996, max=2018,
            step=1, description='XLim:', disabled=False, continuous_update=False,
            orientation='horizontal', readout=True, readout_format='d')
    
    coords_label = Label()
    pt_message = HTML()
    time_label = HTML(value='')
    selected_label = HTML("ID of selected point")
    hover_label = HTML("test value")
    text_brush = HTML(value = 'Selected year range:')
    
    # Create map and controls
    m = ipyleaflet.Map(zoom=5, layout={'height':'400px'}, 
                       center=(3.3890701010382958, -67.32297252983098),dragging=True,
                       close_popup_on_click=False, basemap=ipyleaflet.basemaps.Esri.WorldImagery)

    dc = ipyleaflet.DrawControl(marker={'shapeOptions': {'color': '#ff0000'}}, 
                                polygon={}, circle={}, circlemarker={}, polyline={})
    
    # Table widget
    table_widget = qgrid.show_grid(table, show_toolbar=False)
    
    # Set plots
    # Plot scales.
    lc1_x = bqplot.DateScale(min=datetime.date(xlim.value[0], 2, 1), max=datetime.date(xlim.value[1], 1, 1))
    lc2_y = bqplot.LinearScale(min=ylim.value[0], max=ylim.value[1])
    

    # Main scatter plot for samples
    lc2 = bqplot.Scatter(
        x=[],
        y=[],
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
        interactions={'click': 'select', 'hover': 'tooltip'},
        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
        unselected_style={'opacity': 0.5},
        display_legend=True,
        labels = ['Sample point']
    )

    # Scatter plot for clicked points in map
    lc3 = bqplot.Scatter(
        x=[],
        y=[],
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
        colors=['red'],
        interactions={'click': 'select', 'hover': 'tooltip'},
        selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
        unselected_style={'opacity': 0.5},
        display_legend=True,
        labels = ['Clicked point']
    )
    
    # Pyccd model fit
    lc4 = bqplot.Lines(
        x=[],
        y=[],
        colors=['black'],
        stroke_width=3,
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
    )
    
    # Pyccd model break
    lc5 = bqplot.Scatter(
        x=[],
        y=[],
        marker='triangle-up',
        colors=['red'],
        scales={'x': lc1_x, 'y': lc2_y},
        size=[1,1],
        display_legend=False,
        labels = ['Model Endpoint']
    )

    # Plot axes.
    x_ax1 = bqplot.Axis(label='Date', scale=lc1_x, num_ticks = 6, tick_format='%Y')
    x_ay2 = bqplot.Axis(label='SWIR1', scale=lc2_y, orientation='vertical')

    # Brush selector for figure
    brushintsel = BrushIntervalSelector(scale=lc1_x)

    # Create a figure.
    fig = bqplot.Figure(
        marks=[lc2, lc3, lc4, lc5],
        axes=[x_ax1, x_ay2],
        #layout={'height':'300px', 'width':'800px'},
        layout={'height':'300px', 'width':'100%'},
        #interaction=brushintsel
    )

    def __init__(self, dataframe, navigate):
        Plot_interface.df = dataframe
        Plot_interface.navigate = navigate
        Plot_interface.band_index = 4
        Plot_interface.pyccd_flag = False
        Plot_interface.table = None  
        # Set up database
        conn = sqlite3.connect(Plot_interface.navigate.dbPath)
        Plot_interface.c = conn.cursor()

    @classmethod
    def map_point(self):
        row_id = Plot_interface.df.loc[Plot_interface.df['ID'] == Plot_interface.current_id]
        row_geo = row_id['geometry']
        gjson = ipyleaflet.GeoJSON(data=row_geo[list(row_geo.keys())[0]])
        #gjson = ipyleaflet.GeoJSON(data=Plot_interface.df['geometry'][Plot_interface.current_id])
        Plot_interface.m.center = gjson.data['coordinates'][::-1]
        Plot_interface.m.zoom = 12
        Plot_interface.m.add_layer(gjson)
        
    @classmethod
    def plot_ts(self):
        current_band = Plot_interface.band_list[Plot_interface.band_index]
        row_id = Plot_interface.df.loc[Plot_interface.df['ID'] == Plot_interface.current_id]
        row_geo = row_id['geometry']
        row_geo_dict = row_geo[list(row_geo.keys())[0]]
        point_ts = get_full_collection(row_geo_dict['coordinates'], 
                                      year_range, doy_range) 
        point_df = get_df(point_ts, row_geo_dict['coordinates'], 
                         current_band).dropna()        
        #point_ts = get_full_collection(Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'], 
        #                               year_range, doy_range)
        #point_df = get_df(point_ts, Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'], 
        #                  current_band).dropna()
        Plot_interface.lc2.x = point_df['datetime']
        Plot_interface.lc2.y = point_df[current_band]
        
    # Go back or forth between sample points
    def advance(b):
        # Plot point in map
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        Plot_interface.lc5.display_legend=False
        Plot_interface.pyccd_flag = False
        Plot_interface.current_id += 1
        Plot_interface.pt_message.value = "Point ID: {}".format(Plot_interface.current_id)
#         print(Plot_interface.current_id)
        Plot_interface.map_point() # doesnt work!
        Plot_interface.plot_ts()
        Plot_interface.change_table()
        
#         Plot_interface.change_
        
    def change_table():
        # Update the table based on current ID
        
        # Get header
        cursor = Plot_interface.c.execute('select * from sample')
        names = list(map(lambda x: x[0], cursor.description))
        previous_inputs = pd.DataFrame()
        for i, row in enumerate(Plot_interface.c.execute("SELECT * FROM sample WHERE id = '%s'" % Plot_interface.current_id)):
            previous_inputs[i] = row
        #print(previous_inputs)
        previous_inputs = previous_inputs.T
        if previous_inputs.shape[0] > 0:
            previous_inputs.columns = names
        Plot_interface.table_widget.df = previous_inputs
        #Plot_interface.table = previous_inputs
        #display(previous_inputs)
        
    def on_band_selection(change):        
        new_band = change['new']
        #global band_index
        band_index = change['owner'].index
        Plot_interface.band_index = band_index
        #global current_id
        #gjson = ipyleaflet.GeoJSON(data=Plot_interface.df['geometry'][Plot_interface.current_id])
        #point_ts = get_full_collection(Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'], 
        #                              year_range, doy_range)
        #point_df = get_df(point_ts, Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'], 
        #                  new_band).dropna()
        row_id = Plot_interface.df.loc[Plot_interface.df['ID'] == Plot_interface.current_id]
        row_geo = row_id['geometry']
        row_geo_dict = row_geo[list(row_geo.keys())[0]]
        point_ts = get_full_collection(row_geo_dict['coordinates'], 
                                      year_range, doy_range) 
        point_df = get_df(point_ts, row_geo_dict['coordinates'], 
                         new_band).dropna()  
        #gjson = ipyleaflet.GeoJSON(data=row_geo_dict)

        Plot_interface.lc2.x = point_df['datetime']
        Plot_interface.lc2.y = point_df[new_band]
        Plot_interface.x_ay2.label = new_band
        Plot_interface.lc4.x = []
        Plot_interface.lc4.y = []
        Plot_interface.lc5.x = []
        Plot_interface.lc5.y = []
        # TODO: This should not have to be ran again
        #if pyccd_flag:
        if Plot_interface.pyccd_flag:
            Plot_interface.run_pyccd(0)
        
    def change_yaxis(value):
        Plot_interface.lc2_y.min = Plot_interface.ylim.value[0]
        Plot_interface.lc2_y.max = Plot_interface.ylim.value[1]

    def change_xaxis(value):
        Plot_interface.lc1_x.min = datetime.date(Plot_interface.xlim.value[0], 2, 1)
        Plot_interface.lc1_x.max = datetime.date(Plot_interface.xlim.value[1], 2, 1)
    
    def hover_event(self, target):
        Plot_interface.hover_label.value = str(target['data']['x'])
    
    def click_event(self, target):
        pt_index = target['data']['index']
        # Plot TS TODO
        row_id = Plot_interface.df.loc[Plot_interface.df['ID'] == Plot_interface.current_id]
        row_geo = row_id['geometry']
        row_geo_dict = row_geo[list(row_geo.keys())[0]]
        point_ts = get_full_collection(row_geo_dict['coordinates'], 
                                      year_range, doy_range) 
        point_df = get_df(point_ts, row_geo_dict['coordinates'], 
                         Plot_interface.band_list[Plot_interface.band_index]).dropna()  
        gjson = ipyleaflet.GeoJSON(data=row_geo_dict)
#         gjson = ipyleaflet.GeoJSON(data=Plot_interface.df['geometry'][Plot_interface.current_id])
#         point_ts = get_full_collection(Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'], 
#                                        year_range, doy_range)
#         point_df = get_df(point_ts, Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'], 
#                           'SWIR1').dropna()
        # Find clicked image. .values needed to access the nth element of that list instead of indexing by ID
        image_id = point_df['id'].values[pt_index]
        selected_image = ee.Image(point_ts.filterMetadata('system:index', 'equals', image_id).first())
        tile_url = GetTileLayerUrl(selected_image.visualize(min=0, max=6000, bands= ['NIR', 'SWIR1', 'RED']))
        Plot_interface.m.add_layer(ipyleaflet.TileLayer(url=tile_url))
        #selected_label.value = image_id

    
    # Plotting pyccd
    def plot_pyccd(results, df, band, plotband, dates, yl, ylabel):
        mask = np.array(results['processing_mask']).astype(np.bool_)
        predicted_values = []
        prediction_dates = []
        break_dates = []
        start_dates = []

        #print(results)
        for num, result in enumerate(results['change_models']):
            days = np.arange(result['start_day'], result['end_day'] + 1)
            prediction_dates.append(days)
            break_dates.append(result['break_day'])
            start_dates.append(result['start_day'])
            intercept = result[list(result.keys())[6+band]]['intercept']
            coef = result[list(result.keys())[6+band]]['coefficients']

            predicted_values.append(intercept + coef[0] * days +
                                    coef[1]*np.cos(days*1*2*np.pi/365.25) + coef[2]*np.sin(days*1*2*np.pi/365.25) +
                                    coef[3]*np.cos(days*2*2*np.pi/365.25) + coef[4]*np.sin(days*2*2*np.pi/365.25) +
                                    coef[5]*np.cos(days*3*2*np.pi/365.25) + coef[6]*np.sin(days*3*2*np.pi/365.25))

        num_breaks = len(break_dates)

        # eric here
        break_y = [plotband[dates == i][0] for i in break_dates]

        #break_y = [0] * num_breaks
        break_dates_plot = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in break_dates]

        plot_dates = np.array([datetime.datetime.fromordinal(i) for i in (dates)])

        # Predicted curves
        all_dates = []
        all_preds = []
        for _preddate, _predvalue in zip(prediction_dates, predicted_values):
            all_dates.append(_preddate)
            all_preds.append(_predvalue)

        all_preds = [item for sublist in all_preds for item in sublist]
        all_dates = [item for sublist in all_dates for item in sublist]

        date_ord = [datetime.datetime.fromordinal(i).strftime('%Y-%m-%d %H:%M:%S.%f') for i in all_dates]
        _x = np.array(date_ord, dtype='datetime64')
        _y = all_preds
        Plot_interface.lc4.x = _x
        Plot_interface.lc4.y = _y
        Plot_interface.lc5.x = np.array(break_dates_plot, dtype='datetime64')
        Plot_interface.lc5.y = break_y

    # Run pyccd
    def run_pyccd(b):
        # Run pyCCD on current point
        Plot_interface.pyccd_flag = True
        
        # Display the legend
        Plot_interface.lc5.display_legend=True
        # Plot TS
        name_list = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2','THERMAL', 'pixel_qa']
        point_ts = get_full_collection(Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'], 
                                       year_range, doy_range).select(name_list)

        # Do PyCCD
        pointGeo = ee.Geometry.Point(Plot_interface.df['geometry'][Plot_interface.current_id]['coordinates'])
        dfPyCCD = make_df_pyccd(point_ts, pointGeo)

        dfPyCCD = dfPyCCD.sort_values('time')

        dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'].isnull()] = 4
        dfPyCCD[df.isnull()] = 0
        dfPyCCD['pixel_qa'][dfPyCCD['pixel_qa'] > 4] = 0

        #TODO: Paramaterize everything
        params = {'QA_BITPACKED': False,
                  'QA_FILL': 255,
                  'QA_CLEAR': 0,
                  'QA_WATER': 1,
                  'QA_SHADOW': 2,
                  'QA_SNOW': 3,
                  'QA_CLOUD': 4}

        dates = np.array(dfPyCCD['time'])
        blues = np.array(dfPyCCD['BLUE'])
        greens = np.array(dfPyCCD['GREEN'])
        reds = np.array(dfPyCCD['RED'])
        nirs = np.array(dfPyCCD['NIR'])
        swir1s = np.array(dfPyCCD['SWIR1'])
        swir2s = np.array(dfPyCCD['SWIR2'])
        thermals = np.array(dfPyCCD['THERMAL'])
        qas = np.array(dfPyCCD['pixel_qa'])
        results = ccd.detect(dates, blues, greens, reds, nirs, swir1s, swir2s, thermals, qas, params=params)

        band_names = ['Blue SR', 'Green SR', 'Red SR', 'NIR SR', 'SWIR1 SR', 'SWIR2 SR','THERMAL']
        plotlabel = band_names[Plot_interface.band_index] 

        plot_arrays = [blues, greens, reds, nirs, swir1s, swir2s]
        plotband = plot_arrays[Plot_interface.band_index]
        Plot_interface.plot_pyccd(results, Plot_interface.df, Plot_interface.band_index, 
                                   plotband, dates, (0, 4000), 'PyCCD Results')
        
    ylim.observe(change_yaxis)
    xlim.observe(change_xaxis)
    next_pt.on_click(advance)
    pyccd_button.on_click(run_pyccd)
    band_selector.observe(on_band_selection, names='value')
    
    lc2.on_element_click(click_event)
    lc2.tooltip=hover_label
    lc2.on_hover(hover_event)

    lc3.on_element_click(click_event)
    lc3.tooltip=hover_label
    lc3.on_hover(hover_event)
    
    # Set up map and widgets, plot!
#     dc.on_draw(handle_draw)
#     m.add_control(dc)
#     m.on_interaction(handle_interaction)

In [15]:
newInterface = interface(dbPath)
newPlot = Plot_interface(df, newInterface)

navigate_samples = HBox(children=[newPlot.previous_pt, newPlot.next_pt])
load_data = HBox(children=[newInterface.load_button, newInterface.valid_load])
tab0 = VBox(children=[newInterface.spreadsheet, newInterface.spreadName, newInterface.sampleWidget,
                      load_data, navigate_samples, newPlot.pyccd_button])

tab1 = VBox(children=[newInterface.years,
                      newInterface.lc,
                      newInterface.second_class_drop,
                      newInterface.third_class_drop,
                      newInterface.notes
                      ])
tab2 = VBox(children=[newPlot.band_selector, newPlot.ylim, newPlot.xlim])
tab3 = VBox(children=[newInterface.confidence,
                      newInterface.save_button,
                      newInterface.valid])
tab4 = VBox(children=[newPlot.table_widget])

tab = widgets.Tab(children=[tab0, tab1, tab2, tab3, tab4], layout=Layout(width='100%', min_height='270px', max_height='270px'))
tab.set_title(0, 'Overview')
tab.set_title(1, 'Classification')
tab.set_title(2, 'Visualization')
tab.set_title(3, 'Interpretation')
tab.set_title(4, 'Table')
fig = VBox(children=[newPlot.m, newPlot.fig])
tabfig = VBox(children=[tab, fig])
display(tabfig)

#MEaSUREs-9dc49ca010f9.json
#MEaSUREs_ELB
#users/parevalo_bu/sample_13_15_labels_pts

In [ ]:
display(HTML('<style>.container { width:100% !important; }</style>'))

In [62]:
testing = {14: {'geodesic': True, 'type': 'Point', 'coordinates': [-69.41430227174551, 0.8219715380588426]}}
list(testing.keys())[0]

14